In [1]:
import sys, os
import argparse
import time
from Deconvolution.BLADE import Framework
import numpy as np
from numpy import transpose as t
import itertools
import pickle
from scipy.optimize import nnls
from scipy.stats import gaussian_kde
from matplotlib.colors import LogNorm
from sklearn.svm import SVR
from sklearn.svm import NuSVR

from sklearn.metrics import mean_squared_error as mse
import pandas as pd
from tqdm import trange,tqdm
# modules for visualization
import qgrid
from matplotlib import pyplot as plt
import seaborn as sns
import anndata as ad
import scanpy as sc
import scanorama
from sklearn.model_selection import LeaveOneOut,StratifiedKFold,KFold
import cycler

import warnings
warnings.filterwarnings('ignore')

In [2]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, facecolor='white')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.5 scipy==1.8.0 pandas==1.4.3 scikit-learn==1.0.2 statsmodels==0.13.2 python-igraph==0.9.9 louvain==0.7.1 pynndescent==0.5.7


In [3]:
def getloclist(wd,keyword=["top","marker","DEG"]):
    loc_list = []
    for root, dirs, files in os.walk(wd):
        for file in files:
            for key in keyword:
                if key in file:
                    loc = os.path.join(root, file)
                    loc_list.append(loc) #get file location
                    break
    return loc_list

In [4]:
def record_time():
    t = time.localtime()
    current_time = time.strftime("%Y-%m-%d %H:%M:%S", t)
    print(current_time)

In [5]:
record_time()

2022-07-28 16:09:17


### DEPRECATED BLOCK
### Prepare scRNA signature file for CIBERSORTx input
Run BayesPrism first to get signature.csv file in place

In [7]:
def prepare_CIBERSORTx(path_label,path_signature):
def prepare_CIBERSORTx(path_label,path_signature):
    labels = pd.read_csv(path_label+"cellcategory_simple.csv",index_col=0)
    dict_label = labels.to_dict()['cell_category']
    scRNAref =  pd.read_csv(path_signature,index_col=0)
    scRNAref.columns = scRNAref.columns.map(dict_label)
    scRNAref.to_csv(path_signature.split(".")[0]+"_toCIBERSORTx.tsv",sep='\t')
    print("upload file:",path_signature.split(".")[0]+"_toCIBERSORTx.tsv","to CIBERSORTx as input")
    # see preapare_BLADE for test set, signature file from runBP 

In [146]:
# CAUTIONS! this is not part of the script but you need to run it in notebook after the pipeline script finishes
# list_CIBERSORTx_signature = getloclist("/home/cke/PseudoBulk/Results/",keyword=["_signature.tsv"])
list_CIBERSORTx_signature = getloclist("/home/cke/Real/TCGAResults/",keyword=["_signature.tsv"])

for i in list_CIBERSORTx_signature:
    prepare_CIBERSORTx("/home/cke/Puram/scRNAlabels/",i)

upload file: /home/cke/Real/TCGAResults/fullpipeline_real_Jul28_real_top100markers_signature_toCIBERSORTx.tsv to CIBERSORTx as input
upload file: /home/cke/Real/TCGAResults/fullpipeline_real_Jul28_real_top50DEGs_signature_toCIBERSORTx.tsv to CIBERSORTx as input
upload file: /home/cke/Real/TCGAResults/fullpipeline_real_Jul28_real_top50markers_signature_toCIBERSORTx.tsv to CIBERSORTx as input
upload file: /home/cke/Real/TCGAResults/fullpipeline_real_Jul28_real_top100DEGs_signature_toCIBERSORTx.tsv to CIBERSORTx as input
upload file: /home/cke/Real/TCGAResults/fullpipeline_real_Jul28_real_top20markers_signature_toCIBERSORTx.tsv to CIBERSORTx as input
upload file: /home/cke/Real/TCGAResults/archive/test_fullpipeline_real_Jul21_real_top20markers_signature_toCIBERSORTx.tsv to CIBERSORTx as input
upload file: /home/cke/Real/TCGAResults/archive/test_fullpipeline_real_Jul21_real_top50markers_signature_toCIBERSORTx.tsv to CIBERSORTx as input
upload file: /home/cke/Real/TCGAResults/archive/test_f

/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed: 19.9min
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/pyth

## File preparation
### Pseudobulk: Cross-validation of dataset

In [34]:
def prepare_BLADE(final_adata_mean,final_adata,mode,out):
    # Leave-one-out CV
#prepare var and mean signature matrix for BLADE
    merge_celltype = pd.merge(final_adata_mean.to_df(),final_adata.obs,left_index=True,right_index=True)
    if mode == 'real':
        counts_Puram_mean = merge_celltype.groupby(['Cell_type']).aggregate(np.mean).fillna(0)[final_adata_mean.to_df().columns]
        counts_Puram_std = merge_celltype.groupby(['Cell_type']).aggregate(np.std).fillna(0)[final_adata_mean.to_df().columns]
        counts_Puram_mean.T.to_csv(out+mode+"_mean.tsv",sep='\t')
        #save signature matrices for TCGA deconv
        counts_Puram_std.T.to_csv(out+mode+"_std.tsv",sep='\t')
        list_LOT = []

    elif mode == 'pseudobulk':
        merge_celltype_pseudobulk = merge_celltype.groupby(['batch']).sum()
        sample = merge_celltype_pseudobulk.index.tolist()
        list_LOT = []
        for train_index, test_index in LeaveOneOut().split(sample):
            print("LOT TRAIN:", train_index, "TEST:", test_index)
            train_sample = merge_celltype_pseudobulk.iloc[train_index,].index.tolist()
            train = merge_celltype[merge_celltype['batch'].isin(train_sample)]
            test_sample = merge_celltype_pseudobulk.iloc[test_index,].index.tolist()
            list_LOT.append("".join(str(i) for i in test_sample))
            print("Leaving out: ","".join(str(i) for i in test_sample))
            test = merge_celltype_pseudobulk[merge_celltype_pseudobulk.index.isin(test_sample)]
            counts_Puram_mean = train.groupby(['Cell_type']).aggregate(np.mean).fillna(0)[final_adata_mean.to_df().columns]
            counts_Puram_std = train.groupby(['Cell_type']).aggregate(np.std).fillna(0)[final_adata_mean.to_df().columns]
            counts_Puram_mean.T.to_csv(out+mode+"_LOT"+"".join(str(i) for i in test_sample)+"_mean.tsv",sep='\t')
            counts_Puram_std.T.to_csv(out+mode+"_LOT"+"".join(str(i) for i in test_sample)+"_std.tsv",sep='\t')
            # save the leftout testset for all methods
            test.iloc[:,:final_adata.to_df().shape[1]].to_csv(out+mode+"_LOT"+"".join(str(i) for i in test_sample)+"_test.tsv",sep='\t')
            # for CIBERSORTx
            test.iloc[:,:final_adata.to_df().shape[1]].T.to_csv(out+mode+"_LOT"+"".join(str(i) for i in test_sample)+"_test_transpose.tsv",sep='\t')
    return list_LOT

In [35]:
def prepare_BLADE_kfoldCV(final_adata_mean,final_adata,mode,out,n_splits=5):
    #stratified n-fold CV
#prepare var and mean signature matrix for BLADE
    merge_celltype = pd.merge(final_adata_mean.to_df(),final_adata.obs,left_index=True,right_index=True)
    if mode == 'real':
        counts_Puram_mean = merge_celltype.groupby(['Cell_type']).aggregate(np.mean).fillna(0)[final_adata_mean.to_df().columns]
        counts_Puram_std = merge_celltype.groupby(['Cell_type']).aggregate(np.std).fillna(0)[final_adata_mean.to_df().columns]
        counts_Puram_mean.T.to_csv(out+mode+"_mean.tsv",sep='\t')
        #save signature matrices for TCGA deconv
        counts_Puram_std.T.to_csv(out+mode+"_std.tsv",sep='\t')
        list_CV = []

    elif mode == 'pseudobulk':
        singlecells = merge_celltype.index.tolist()
        batch = merge_celltype['batch'].tolist()
        list_CV = []
        count = 1
        for train_index, test_index in StratifiedKFold(n_splits=n_splits).split(singlecells,batch):
            print("CV TRAIN:",count , len(train_index), "TEST:", len(test_index))
            train_sc = merge_celltype.iloc[train_index,].index.tolist()
            train = merge_celltype.loc[train_sc,]
            test_sc = merge_celltype.iloc[test_index,].index.tolist()
            list_CV.append(str(count))
            test = merge_celltype.loc[test_sc,].groupby(['batch']).sum()
            counts_Puram_mean = train.groupby(['Cell_type']).aggregate(np.mean).fillna(0)[final_adata_mean.to_df().columns]
            counts_Puram_std = train.groupby(['Cell_type']).aggregate(np.std).fillna(0)[final_adata_mean.to_df().columns]
            counts_Puram_mean.T.to_csv(out+mode+"_CV"+str(count)+"_mean.tsv",sep='\t')
            counts_Puram_std.T.to_csv(out+mode+"_CV"+str(count)+"_std.tsv",sep='\t')
            # save the leftout testset for all methods
            test.iloc[:,:final_adata.to_df().shape[1]].to_csv(out+mode+"_CV"+str(count)+"_test.tsv",sep='\t')
            # for CIBERSORTx
            test.iloc[:,:final_adata.to_df().shape[1]].T.to_csv(out+mode+"_CV"+str(count)+"_test_transpose.tsv",sep='\t')
            count+=1
    return list_CV

In [56]:
list_CV = prepare_BLADE_kfoldCV(final_adata_mean,final_adata,'pseudobulk',"/home/cke/PseudoBulk/InputToWrapper/")

CV TRAIN: 1 3587 TEST: 897
CV TRAIN: 2 3587 TEST: 897
CV TRAIN: 3 3587 TEST: 897
CV TRAIN: 4 3587 TEST: 897
CV TRAIN: 5 3588 TEST: 896


In [153]:
list_CV = prepare_BLADE(final_adata_mean,final_adata,'pseudobulk',"/home/cke/PseudoBulk/InputToWrapper/") 

LOT TRAIN: [1 2 3 4 5 6 7 8] TEST: [0]
Leaving out:  6
LOT TRAIN: [0 2 3 4 5 6 7 8] TEST: [1]
Leaving out:  16
LOT TRAIN: [0 1 3 4 5 6 7 8] TEST: [2]
Leaving out:  17
LOT TRAIN: [0 1 2 4 5 6 7 8] TEST: [3]
Leaving out:  18
LOT TRAIN: [0 1 2 3 5 6 7 8] TEST: [4]
Leaving out:  20
LOT TRAIN: [0 1 2 3 4 6 7 8] TEST: [5]
Leaving out:  22
LOT TRAIN: [0 1 2 3 4 5 7 8] TEST: [6]
Leaving out:  25
LOT TRAIN: [0 1 2 3 4 5 6 8] TEST: [7]
Leaving out:  26
LOT TRAIN: [0 1 2 3 4 5 6 7] TEST: [8]
Leaving out:  28


In [123]:
list_CV = prepare_BLADE(final_adata_mean,final_adata,'real',"/home/cke/Real/InputToWrapper/") 

In [36]:
def prepare_Rdeconv(final_adata,mode,out):
    merge_sample = pd.merge(final_adata.to_df(),final_adata.obs,left_index=True,right_index=True)
    if mode == 'real':
        scRNA_input = final_adata.to_df().loc[:,final_adata.to_df().columns]
        scRNA_input.to_csv(out+mode+"_scRNAref.tsv",sep='\t')
        list_LOT = []
        
    elif mode == 'pseudobulk':
        merge_sample_pseudobulk = merge_sample.groupby(['batch']).sum()
        sample = merge_sample_pseudobulk.index.tolist()
        list_LOT = []
        for train_index, test_index in LeaveOneOut().split(sample):
            print("LOT TRAIN:", train_index, "TEST:", test_index)
            train_sample = merge_sample_pseudobulk.iloc[train_index,].index.tolist()
            train = merge_sample[merge_sample['batch'].isin(train_sample)]
            test_sample = merge_sample_pseudobulk.iloc[test_index,].index.tolist()
            list_LOT.append("".join(str(i) for i in test_sample))
            print("Leaving out: ","".join(str(i) for i in test_sample))
            test = merge_sample_pseudobulk[merge_sample_pseudobulk.index.isin(test_sample)]
            train.iloc[:,:final_adata.to_df().shape[1]].to_csv(out+mode+"_LOT"+"".join(str(i) for i in test_sample)+"_scRNAtrain.tsv",sep='\t')
            test.iloc[:,:final_adata.to_df().shape[1]].to_csv(out+mode+"_LOT"+"".join(str(i) for i in test_sample)+"_test.tsv",sep='\t')
            # for CIBERSORTx
            test.iloc[:,:final_adata.to_df().shape[1]].T.to_csv(out+mode+"_LOT"+"".join(str(i) for i in test_sample)+"_test_transpose.tsv",sep='\t')
    return list_LOT

In [37]:
def prepare_Rdeconv_kfoldCV(final_adata,mode,out,n_splits=5):
    merge_sample = pd.merge(final_adata.to_df(),final_adata.obs,left_index=True,right_index=True)
    if mode == 'real':
        scRNA_input = final_adata.to_df().loc[:,final_adata.to_df().columns]
        scRNA_input.to_csv(out+mode+"_scRNAref.tsv",sep='\t')
        list_CV = []
        
    elif mode == 'pseudobulk':
        singlecells = merge_celltype.index.tolist()
        batch = merge_celltype['batch'].tolist()
        list_CV = []
        count = 1
        for train_index, test_index in StratifiedKFold(n_splits=n_splits).split(singlecells,batch):
            print("CV TRAIN:",count , len(train_index), "TEST:", len(test_index))
            train_sc = merge_celltype.iloc[train_index,].index.tolist()
            train = merge_celltype.loc[train_sc,]
            test_sc = merge_celltype.iloc[test_index,].index.tolist()
            list_CV.append(str(count))
            test = merge_celltype.loc[test_sc,].groupby(['batch']).sum()
            train.iloc[:,:final_adata.to_df().shape[1]].to_csv(out+mode+"_CV"+str(count)+"_scRNAtrain.tsv",sep='\t')
            test.iloc[:,:final_adata.to_df().shape[1]].to_csv(out+mode+"_CV"+str(count)+"_test.tsv",sep='\t')
            # for CIBERSORTx
            test.iloc[:,:final_adata.to_df().shape[1]].T.to_csv(out+mode+"_CV"+str(count)+"_test_transpose.tsv",sep='\t')
            count+=1
    return list_CV

In [52]:
list_CV = prepare_Rdeconv_kfoldCV(final_adata,'pseudobulk',"/home/cke/PseudoBulk/InputToWrapper/")

CV TRAIN: 1 3587 TEST: 897
CV TRAIN: 2 3587 TEST: 897
CV TRAIN: 3 3587 TEST: 897
CV TRAIN: 4 3587 TEST: 897
CV TRAIN: 5 3588 TEST: 896


In [124]:
list_CV = prepare_Rdeconv(final_adata,'pseudobulk',"/home/cke/PseudoBulk/InputToWrapper/")

LOT TRAIN: [1 2 3 4 5 6 7 8] TEST: [0]
Leaving out:  [6]
LOT TRAIN: [0 2 3 4 5 6 7 8] TEST: [1]
Leaving out:  [16]
LOT TRAIN: [0 1 3 4 5 6 7 8] TEST: [2]
Leaving out:  [17]
LOT TRAIN: [0 1 2 4 5 6 7 8] TEST: [3]
Leaving out:  [18]
LOT TRAIN: [0 1 2 3 5 6 7 8] TEST: [4]
Leaving out:  [20]
LOT TRAIN: [0 1 2 3 4 6 7 8] TEST: [5]
Leaving out:  [22]
LOT TRAIN: [0 1 2 3 4 5 7 8] TEST: [6]
Leaving out:  [25]
LOT TRAIN: [0 1 2 3 4 5 6 8] TEST: [7]
Leaving out:  [26]
LOT TRAIN: [0 1 2 3 4 5 6 7] TEST: [8]
Leaving out:  [28]


In [125]:
list_CV = prepare_Rdeconv(final_adata,'real',"/home/cke/Real/InputToWrapper/")

### Running each method in cmd

In [38]:
def run_cmd(mode,out,out_res,dict_FS,folder_marker,methods,
            test_sample=False,path_label="home/cke/Puram/scRNAlabels/",
            path_bulk=False,name="unnamed_job"):
    # "out" is folder where input files prepared in last steps
    if mode == 'real':
        for FS_setup, marker_file in tqdm(dict_FS.items()):
            time.sleep(30)
            print(mode,"mode: now with feature selection setup: \r",FS_setup)
            record_time()
            if 'MuSiC' in methods:
                cmd_MuSiC = "".join(["Rscript /home/cke/runscripts/runMuSiC.r ", 
                                 out,mode,"_scRNAref.tsv ", # scRNA signature matrix
                                path_bulk," ", # real bulk RNAseq matrix
                                 path_label," ", 
                              marker_file," ",
                               name+"_"+mode+"_"+FS_setup+" ", # job name FS_setup
                              out_res+"MuSiC/"," &", # output frac folder
                                ])
                print("now running in cmd: \r",cmd_MuSiC)
                os.system(cmd_MuSiC)
            if 'BP' in methods:
                cmd_BP = "".join(["Rscript /home/cke/runscripts/runBP.r ", 
                                 out,mode,"_scRNAref.tsv ", # signature matrix
                                path_bulk," ", # testset pseudobulk matrix
                                 path_label," ",
                              marker_file," ",
                               name+"_"+mode+"_"+FS_setup+" ", # job name FS_setup
                              out_res+"BayesPrism/"," ", # output frac folder
                              out_res," &" # output CIBERSORTx prelim signature
                                ])
                print("now running in cmd: \r",cmd_BP)
                os.system(cmd_BP)
        # for BLADE, feature selection is done within wrapper
        # R is lame, plz use python to develop new tool :)
        if 'BLADE' in methods:
            cmd_BLADE = "".join(["python /home/cke/runscripts/runBLADE.py ", 
                             out,mode,"_std.tsv ", # std signature matrix
                             out,mode,"_mean.tsv ", # mean signature matrix
                            path_bulk," ", # testset pseudobulk matrix
                             out_res+"BLADE/"," ", # output folder
                             "--folder_marker ",folder_marker," ", # marker folder
                             "--name ",name+"_"+mode," &" # job name, background run
                            ])
            print("now running in cmd: \r",cmd_BLADE)
            os.system(cmd_BLADE)
    elif mode == "pseudobulk":
        for FS_setup, marker_file in tqdm(dict_FS.items()):
            time.sleep(30)
            record_time()
            print(mode,"mode: now with feature selection setup: \r",FS_setup)
            if 'MuSiC' in methods:
                cmd_MuSiC = "".join(["Rscript /home/cke/runscripts/runMuSiC.r ", 
                                 out,mode,"_LOT",str(test_sample),"_scRNAtrain.tsv ", # signature matrix
                                out,mode,"_LOT",str(test_sample),"_test.tsv ", # testset pseudobulk matrix
                                 path_label," ",
                              marker_file," ",
                               name+"_"+mode+"_"+FS_setup+"_"+str(test_sample)+"LOT ", # job name, FS_setup
                              out_res+"MuSiC/"," &", # output frac folder
                                ])
                print("now running in cmd: \r",cmd_MuSiC)
                os.system(cmd_MuSiC)
            if 'BP' in methods:   
                cmd_BP = "".join(["Rscript /home/cke/runscripts/runBP.r ", 
                                 out,mode,"_LOT",str(test_sample),"_scRNAtrain.tsv ", # signature matrix
                                out,mode,"_LOT",str(test_sample),"_test.tsv ", # testset pseudobulk matrix
                                 path_label," ",
                              marker_file," ",
                               name+"_"+mode+"_"+FS_setup+"_"+str(test_sample)+"LOT ", # job name, FS_setup
                              out_res+"BayesPrism/"," ", # output frac folder
                              out_res," &" # output CIBERSORTx prelim signature
                                ])
                print("now running in cmd: \r",cmd_BP)
                os.system(cmd_BP)
        # for BLADE, feature selection is done within wrapper
        # R is lame, plz use python to develop new tool :)
        if 'BLADE' in methods:
            cmd_BLADE = "".join(["python /home/cke/runscripts/runBLADE.py ", 
                             out,mode,"_LOT",str(test_sample),"_std.tsv ", # std signature matrix
                             out,mode,"_LOT"+str(test_sample),"_mean.tsv ", # mean signature matrix
                            out,mode,"_LOT",str(test_sample),"_test.tsv ", # testset pseudobulk matrix
                             out_res+"BLADE/"," ", # output folder
                             "--folder_marker ",folder_marker," ", # marker folder
                             "--name ",name+"_"+mode+"_"+"LOT"+str(test_sample)," &" # job name, background run
                            ])
            print("now running in cmd: \r",cmd_BLADE)
            os.system(cmd_BLADE)

In [39]:
def run_cmd_noFS(mode,out,out_res,methods,test_sample=False,
                 path_label="home/cke/Puram/scRNAlabels/",
                 path_bulk=False,name="unnamed_noFS_job"):
    # "out" is folder where input files prepared in last steps
    marker_file = "noFS"
    if mode == 'real':
        print(mode,"mode: no feature selection\r")
        record_time()
        if 'MuSiC' in methods:
            cmd_MuSiC = "".join(["Rscript /home/cke/runscripts/runMuSiC.r ", 
                             out,mode,"_scRNAref.tsv ", # scRNA signature matrix
                            path_bulk," ", # real bulk RNAseq matrix
                             path_label," ", 
                          marker_file," ",
                           name+"_"+mode+"_"+"noFS"+" ", # job name, FS_setup
                          out_res+"MuSiC/"," &", # output frac folder
                            ])
            print("now running in cmd: \r",cmd_MuSiC)
            os.system(cmd_MuSiC)
        if 'BP' in methods: 
            cmd_BP = "".join(["Rscript /home/cke/runscripts/runBP.r ", 
                             out,mode,"_scRNAref.tsv ", # signature matrix
                            path_bulk," ", # testset pseudobulk matrix
                             path_label," ",
                          marker_file," ",
                           name+"_"+mode+"_"+"noFS"+" ", # job name FS_setup
                          out_res+"BayesPrism/"," ", # output frac folder
                          out_res," &" # output CIBERSORTx prelim signature
                            ])
            print("now running in cmd: \r",cmd_BP)
            os.system(cmd_BP)
        # for BLADE, feature selection is done within wrapper
        # R is lame, plz use python to develop new tool :)
        if 'BLADE' in methods:
            cmd_BLADE = "".join(["python /home/cke/runscripts/runBLADE.py ", 
                             out,mode,"_std.tsv ", # std signature matrix
                             out,mode,"_mean.tsv ", # mean signature matrix
                            path_bulk," ", # testset pseudobulk matrix
                             out_res+"BLADE/"," ", # output folder
                             "--name ",name+"_"+mode,"_noFS &" # job name, background run
                            ])
            print("now running in cmd: \r",cmd_BLADE)
            os.system(cmd_BLADE)
    elif mode == "pseudobulk":
        print(mode,"mode: no feature selection\r")
        record_time()
        if 'MuSiC' in methods:
            cmd_MuSiC = "".join(["Rscript /home/cke/runscripts/runMuSiC.r ", 
                             out,mode,"_LOT",str(test_sample),"_scRNAtrain.tsv ", # signature matrix
                            out,mode,"_LOT",str(test_sample),"_test.tsv ", # testset pseudobulk matrix
                             path_label," ",
                          marker_file," ",
                           name+"_"+mode+"_"+"noFS"+"_"+str(test_sample)+"_LOT ", # job name, FS_setup
                          out_res+"MuSiC/"," &", # output frac folder
                            ])
            print("now running in cmd: \r",cmd_MuSiC)
            os.system(cmd_MuSiC)
        if 'BP' in methods: 
            cmd_BP = "".join(["Rscript /home/cke/runscripts/runBP.r ", 
                             out,mode,"_LOT",str(test_sample),"_scRNAtrain.tsv ", # signature matrix
                            out,mode,"_LOT",str(test_sample),"_test.tsv ", # testset pseudobulk matrix
                             path_label," ",
                          marker_file," ",
                           name+"_"+mode+"_"+"noFS"+"_"+str(test_sample)+"_LOT ", # job name, FS_setup
                          out_res+"BayesPrism/"," ", # output frac folder
                          out_res," &" # output CIBERSORTx prelim signature
                            ])
            print("now running in cmd: \r",cmd_BP)
            os.system(cmd_BP)
        # for BLADE, feature selection is done within wrapper
        # R is lame, plz use python to develop new tool :)
        if 'BLADE' in methods:
            cmd_BLADE = "".join(["python /home/cke/runscripts/runBLADE.py ", 
                             out,mode,"_LOT",str(test_sample),"_std.tsv ", # std signature matrix
                             out,mode,"_LOT"+str(test_sample),"_mean.tsv ", # mean signature matrix
                            out,mode,"_LOT",str(test_sample),"_test.tsv ", # testset pseudobulk matrix
                             out_res+"BLADE/"," ", # output folder
                             "--name ",name+"_"+mode+"_"+"LOT"+str(test_sample)+"_noFS"," &" # job name, background run
                            ])
            print("now running in cmd: \r",cmd_BLADE)
            os.system(cmd_BLADE)

In [40]:
def main(path_adata,mode,out,out_res,path_label,path_bulk=False,folder_marker=False,
         name="unnamed_job",methods=['MuSiC','BP','BLADE'],keyword=["top","marker","DEG"]):
    # "out" is folder where input files prepared in last steps
#     print(folder_marker)
    record_time()
    final_adata = sc.read_h5ad(path_adata)
    final_adata.uns['log1p']["base"] = None
    final_adata_mean = final_adata.copy()
    sc.pp.log1p(final_adata_mean)
    if folder_marker:
        print("running Feature selection!\r")
        list_markers = getloclist(folder_marker,keyword)
        dict_FS = {}
        for marker_file in list_markers:
#             marker_genes =  pd.read_csv(marker_file,header=None).iloc[0,:]
            dict_FS[os.path.split(marker_file)[1].split("_")[0]] = marker_file
            
        if mode == 'real':
            list_CV = prepare_BLADE(final_adata_mean,final_adata,mode,out)
            list_CV = prepare_Rdeconv(final_adata,mode,out)
            run_cmd(mode,out,out_res,dict_FS,folder_marker,methods,
                    path_label=path_label,path_bulk=path_bulk,name=name)
            
        elif mode == 'pseudobulk':
            list_CV = prepare_BLADE(final_adata_mean,final_adata,mode,out)
            list_CV = prepare_Rdeconv(final_adata,mode,out)
            for test_sample in tqdm(list_CV):
                time.sleep(30)
                run_cmd(mode,out,out_res,dict_FS,folder_marker,methods,
                        test_sample=test_sample,path_label=path_label,name=name)
#                 if True: 
#                     prepare_CIBERSORTx(path_label,path_signature)
    else:
        print("no Feature selection!\r")
        if mode == 'real':
            list_CV = prepare_BLADE(final_adata_mean,final_adata,mode,out)
            list_CV = prepare_Rdeconv(final_adata,mode,out)
            run_cmd_noFS(mode,out,out_res,methods,
                    path_label=path_label,path_bulk=path_bulk,name=name)
        elif mode == 'pseudobulk':
            list_CV = prepare_BLADE(final_adata_mean,final_adata,mode,out)
            list_CV = prepare_Rdeconv(final_adata,mode,out)
            for test_sample in tqdm(list_CV):
                time.sleep(60)
                run_cmd_noFS(mode,out,out_res,methods,
                        test_sample=test_sample,path_label=path_label,name=name)

In [41]:
def parse_args():
    """
        Parses inputs from the commandline.
        :return: inputs as a Namespace object
    """
    parser = argparse.ArgumentParser(description='Generates pipeline')
    # Arguments
    parser.add_argument('path_adata', help='directory of preprocessed raw scRNA anndata object')
    parser.add_argument('mode', help='scheme for data processing', choices=['pseudobulk','real'])
    parser.add_argument('out', help='output CV input directory')
    parser.add_argument('out_res', help='output of decon methods directory')
    parser.add_argument('path_label', help='labels of single-cell type identity directory')
    parser.add_argument('--path_bulk', help='bulk rnaseq data directory',default=False)
    parser.add_argument('--folder_marker', help='the folder where markers is stored',default=False)
    parser.add_argument('--name', help='give this job a name to help remember',default='unnamed_job')
    parser.add_argument('--methods', help='which methods you want to use',default=['MuSiC','BP','BLADE'])
    parser.add_argument('--keyword', help='keyword in marker file name to identify them',default=["top","marker","DEG"])
    return parser.parse_args()

In [ ]:
if __name__ == "__main__":
    args = parse_args()
    path_adata = args.path_adata
    mode = args.mode
    out = args.out
    out_res = args.out_res
    path_label = args.path_label
    path_bulk = args.path_bulk
    name = args.name
    folder_marker = args.folder_marker
    methods = args.methods
    keyword = args.keyword
    if name == "unnamed_job":
        print("You did not name this job!\r")
    else:
        print(name," - pipeline initiated! Welcome, contact author for support: kechanglin1998@hotmail.com\r")

    if mode == "real":
        if path_bulk == False:
            raise ValueError("no bulk RNAseq data input! LOAD UP YOUR AMMO!\r")
        main(path_adata,mode,out,out_res,path_label,path_bulk=path_bulk,folder_marker=folder_marker,
         name=name,methods=methods,keyword=keyword)
    elif mode == 'pseudobulk':
        main(path_adata,mode,out,out_res,path_label,path_bulk=path_bulk,folder_marker=folder_marker,
         name=name,methods=methods,keyword=keyword)
    print("job finished! details of arguments used in this run: ",args)


## Testing code

In [48]:
# testing code block
# real
path_adata = "/home/cke/Puram/Puram_scanpy.h5ad"
mode = "real"
out = "/home/cke/Real/InputToWrapper/"
out_res = "/home/cke/Real/TCGAResults/"
path_label = "/home/cke/Puram/scRNAlabels/"
path_bulk = "/home/cke/TCGA-HNSC.htseq_counts_exp2_symbol_samplexgene.tsv"
name = "fullpipeline_real_Jul28"
folder_marker = "/home/cke/Puram/markers/" 
methods = ['BLADE']
keyword = ["top","marker","DEG"]

In [49]:
# testing code block

if name == "unnamed_job":
    print("You did not name this job!\r")
else:
    print(name," - pipeline initiated! Welcome, contact author for support: kechanglin1998@hotmail.com\r")

if mode == "real":
    if path_bulk == False:
        raise ValueError("no bulk RNAseq data input! LOAD UP YOUR AMMO!\r")
    main(path_adata,mode,out,out_res,path_label,path_bulk=path_bulk,folder_marker=folder_marker,
     name=name,methods=methods,keyword=keyword)
elif mode == 'pseudobulk':
    main(path_adata,mode,out,out_res,path_label,path_bulk=path_bulk,folder_marker=folder_marker,
     name=name,methods=methods,keyword=keyword)
# print("job submitted! details of arguments used in this run: ",args)


fullpipeline_real_Jul28  - pipeline initiated! Welcome, contact author for support: kechanglin1998@hotmail.com
2022-07-28 20:04:24
running Feature selection!


 20%|██████████████████▏                                                                        | 1/5 [00:30<02:00, 30.03s/it]

 top50DEGs
2022-07-28 20:06:05


 40%|████████████████████████████████████▍                                                      | 2/5 [01:00<01:30, 30.03s/it]

 top100markers
2022-07-28 20:06:35


...

Done optimization, elapsed time (min): 268.43551086187364
export to:  /home/cke/Real/TCGAResults/BLADE/fullpipeline_real_Jul28_real_BLADEout_top50DEGs.pickle
now with feature selection setup:  top100markers
Get mean and std exp!
Get common genes!  830
cell types:  10
bulk samples:  546
start BLADE!
all of 830 genes are used for optimization.
All samples are used during the optimization.
Initialization with Support vector regression


[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:   12.8s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   22.7s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   33.1s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   45.9s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:  1.2min
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:  1.7min
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:  2.0min
[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:  2.4min
[Parallel(n_jobs=10)]: Done  93 tasks      | elapsed:  2.7min
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  3.3min
[Parallel(n_jobs=10)]: Done 125 tasks      | elapsed:  3.8min
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:  4.4min
[Parallel(n_jobs=10)]: Done 161 tasks      | elapsed:  5.1min
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:  5.6min
[Parallel(n_jobs=10)]: Done 201 tasks      | elapsed:  7.0min
[Parallel(n_jobs=10)]: Done 222 tasks      | elapsed:  9.4min
[Paralle

No feature filtering is done (fsel = 0)


/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarnin

/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarnin

In [44]:
# testing code block
# pseudobulk
path_adata = "/home/cke/Puram/Puram_scanpy.h5ad"
mode = "pseudobulk"
out = "/home/cke/PseudoBulk/InputToWrapper/"
out_res = "/home/cke/PseudoBulk/Results/"
path_label = "/home/cke/Puram/scRNAlabels/"
path_bulk = False
name = "fullpipeline_pseudobulk_Jul28_LOT"
folder_marker = "/home/cke/Puram/markers/" 
methods = ['BP']
keyword = ["top","marker","DEG"]

In [45]:
# testing code block

if name == "unnamed_job":
    print("You did not name this job!\r")
else:
    print(name," - pipeline initiated! Welcome, contact author for support: kechanglin1998@hotmail.com\r")

if mode == "real":
    if path_bulk == False:
        raise ValueError("no bulk RNAseq data input! LOAD UP YOUR AMMO!\r")
    main(path_adata,mode,out,out_res,path_label,path_bulk=path_bulk,folder_marker=folder_marker,
     name=name,methods=methods,keyword=keyword)
elif mode == 'pseudobulk':
    main(path_adata,mode,out,out_res,path_label,path_bulk=path_bulk,folder_marker=folder_marker,
     name=name,methods=methods,keyword=keyword)
# print("jobs submitted! details of arguments used in this run: ",args)


fullpipeline_pseudobulk_Jul28_LOT  - pipeline initiated! Welcome, contact author for support: kechanglin1998@hotmail.com
2022-07-28 18:26:45
running Feature selection!
LOT TRAIN: [1 2 3 4 5 6 7 8] TEST: [0]
Leaving out:  6
LOT TRAIN: [0 2 3 4 5 6 7 8] TEST: [1]
Leaving out:  16
LOT TRAIN: [0 1 3 4 5 6 7 8] TEST: [2]
Leaving out:  17
LOT TRAIN: [0 1 2 4 5 6 7 8] TEST: [3]
Leaving out:  18
LOT TRAIN: [0 1 2 3 5 6 7 8] TEST: [4]
Leaving out:  20
LOT TRAIN: [0 1 2 3 4 6 7 8] TEST: [5]
Leaving out:  22
LOT TRAIN: [0 1 2 3 4 5 7 8] TEST: [6]
Leaving out:  25
LOT TRAIN: [0 1 2 3 4 5 6 8] TEST: [7]
Leaving out:  26
LOT TRAIN: [0 1 2 3 4 5 6 7] TEST: [8]
Leaving out:  28
LOT TRAIN: [1 2 3 4 5 6 7 8] TEST: [0]
Leaving out:  6
LOT TRAIN: [0 2 3 4 5 6 7 8] TEST: [1]
Leaving out:  16
LOT TRAIN: [0 1 3 4 5 6 7 8] TEST: [2]
Leaving out:  17
LOT TRAIN: [0 1 2 4 5 6 7 8] TEST: [3]
Leaving out:  18
LOT TRAIN: [0 1 2 3 5 6 7 8] TEST: [4]
Leaving out:  20
LOT TRAIN: [0 1 2 3 4 6 7 8] TEST: [5]
Leaving out

 20%|███████████████████▌                                                                              | 1/5 [00:30<02:00, 30.05s/it]

2022-07-28 18:39:11
 top50DEGs
 Rscript /home/cke/runscripts/runBP.r /home/cke/PseudoBulk/InputToWrapper/pseudobulk_LOT6_scRNAtrain.tsv /home/cke/PseudoBulk/InputToWrapper/pseudobulk_LOT6_test.tsv /home/cke/Puram/scRNAlabels/ /home/cke/Puram/markers/top50DEGs_pseudobulk.txt fullpipeline_pseudobulk_Jul28_LOT_pseudobulk_top50DEGs_6LOT /home/cke/PseudoBulk/Results/BayesPrism/ /home/cke/PseudoBulk/Results/ &


Warning message:
replacing previous import ‘gplots::lowess’ by ‘stats::lowess’ when loading ‘TED’ 


[1] "running BayesPrism with following args:"
[1] "/home/cke/PseudoBulk/InputToWrapper/pseudobulk_LOT6_scRNAtrain.tsv"
[2] "/home/cke/PseudoBulk/InputToWrapper/pseudobulk_LOT6_test.tsv"      
[3] "/home/cke/Puram/scRNAlabels/"                                      
[4] "/home/cke/Puram/markers/top50DEGs_pseudobulk.txt"                  
[5] "fullpipeline_pseudobulk_Jul28_LOT_pseudobulk_top50DEGs_6LOT"       
[6] "/home/cke/PseudoBulk/Results/BayesPrism/"                          
[7] "/home/cke/PseudoBulk/Results/"                                     
[8] "/home/cke/Puram/scRNAlabels/"                                      


...

        tumor Fibroblast B cell myocyte other Macrophage T cell Dendritic  Mast
Min.    0.247      0.625  0.056   0.033     0      0.009      0         0 0.004
1st Qu. 0.247      0.625  0.056   0.033     0      0.009      0         0 0.004
Median  0.247      0.625  0.056   0.033     0      0.009      0         0 0.004
Mean    0.247      0.625  0.056   0.033     0      0.009      0         0 0.004
3rd Qu. 0.247      0.625  0.056   0.033     0      0.009      0         0 0.004
Max.    0.247      0.625  0.056   0.033     0      0.009      0         0 0.004
        Endothelial
Min.          0.027
1st Qu.       0.027
Median        0.027
Mean          0.027
3rd Qu.       0.027
Max.          0.027
[1] "only one mixture sample. vst transformation is NOT feasible"



Stopping cluster



[1] "run final sampling"
Start run... This may take a while 


snowfall 1.84-6.2 initialized (using snow 0.4-4): parallel execution on 10 CPUs.


Stopping cluster

Warning messages:
1: In searchCommandline(parallel, cpus = cpus, type = type, socketHosts = socketHosts,  :
  Unknown option on commandline: --file
2: In searchCommandline(parallel, cpus = cpus, type = type, socketHosts = socketHosts,  :
  Unknown option on commandline: --file
3: In searchCommandline(parallel, cpus = cpus, type = type, socketHosts = socketHosts,  :
  Unknown option on commandline: --file


        tumor Fibroblast B cell myocyte other Macrophage T cell Dendritic  Mast
Min.    0.302      0.551  0.076   0.031     0      0.017      0         0 0.006
1st Qu. 0.302      0.551  0.076   0.031     0      0.017      0         0 0.006
Median  0.302      0.551  0.076   0.031     0      0.017      0         0 0.006
Mean    0.302      0.551  0.076   0.031     0      0.017      0         0 0.006
3rd Qu. 0.302      0.551  0.076   0.031     0      0.017      0         0 0.006
Max.    0.302      0.551  0.076   0.031     0      0.017      0         0 0.006
        Endothelial
Min.          0.016
1st Qu.       0.016
Median        0.016
Mean          0.016
3rd Qu.       0.016
Max.          0.016
[1] "only one mixture sample. vst transformation is NOT feasible"


In [27]:
# python "/home/cke/runscripts/Pipeline_Deconv_Run.py" "/home/cke/Puram/Puram_scanpy.h5ad" "pseudobulk" "/home/cke/PseudoBulk/InputToWrapper/" "/home/cke/PseudoBulk/Results/" "/home/cke/Puram/scRNAlabels/"  --name "fullpipeline_Jul29_noFS"

[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:  4.0min

Stopping cluster



[1] "run final sampling"
Start run... This may take a while 


[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:  2.7min
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  4.1min
[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:  4.2min

Stopping cluster



[1] "run final sampling"
Start run... This may take a while 


snowfall 1.84-6.2 initialized (using snow 0.4-4): parallel execution on 10 CPUs.

snowfall 1.84-6.2 initialized (using snow 0.4-4): parallel execution on 10 CPUs.

[Parallel(n_jobs=10)]: Done  93 tasks      | elapsed:  1.3min
snowfall 1.84-6.2 initialized (using snow 0.4-4): parallel execution on 10 CPUs.

[Parallel(n_jobs=10)]: Done 125 tasks      | elapsed:  4.0min
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  1.5min

Stopping cluster



...

No feature filtering is done (fsel = 0)
Done optimization, elapsed time (min): 1.8237806280454
export to:  /home/cke/PseudoBulk/Results/BLADE/fullpipeline_pseudobulk_Jul28_LOT_pseudobulk__LOT28_BLADEout_top20markers.pickle
now with feature selection setup:  top100DEGs
Get mean and std exp!
Get common genes!  876
cell types:  10
bulk samples:  1
start BLADE!
all of 876 genes are used for optimization.
All samples are used during the optimization.
Initialization with Support vector regression
No feature filtering is done (fsel = 0)
Done optimization, elapsed time (min): 2.6677656292915346
export to:  /home/cke/PseudoBulk/Results/BLADE/fullpipeline_pseudobulk_Jul28_LOT_pseudobulk__LOT28_BLADEout_top100DEGs.pickle
now with feature selection setup:  top50markers
Get mean and std exp!
Get common genes!  467
cell types:  10
bulk samples:  1
start BLADE!
all of 467 genes are used for optimization.
All samples are used during the optimization.
Initialization with Support vector regression
No fe

[Parallel(n_jobs=10)]: Done 160 out of 162 | elapsed:  3.9min remaining:    3.0s
[Parallel(n_jobs=10)]: Done 162 out of 162 | elapsed:  6.6min finished


No feature filtering is done (fsel = 0)
Done optimization, elapsed time (min): 6.556673912207286
export to:  /home/cke/PseudoBulk/Results/BLADE/fullpipeline_pseudobulk_Jul28_LOT_pseudobulk__LOT16_BLADEout_top50markers.pickle


[Parallel(n_jobs=10)]: Done 162 out of 162 | elapsed:  3.6min finished


No feature filtering is done (fsel = 0)
Done optimization, elapsed time (min): 2.3176085591316222
export to:  /home/cke/PseudoBulk/Results/BLADE/fullpipeline_pseudobulk_Jul28_LOT_pseudobulk__LOT20_BLADEout_top20markers.pickle
now with feature selection setup:  top100DEGs
Get mean and std exp!
Get common genes!  876
cell types:  10
bulk samples:  1
start BLADE!
all of 876 genes are used for optimization.
All samples are used during the optimization.
Initialization with Support vector regression
No feature filtering is done (fsel = 0)
Done optimization, elapsed time (min): 1.4670257568359375
export to:  /home/cke/PseudoBulk/Results/BLADE/fullpipeline_pseudobulk_Jul28_LOT_pseudobulk__LOT20_BLADEout_top100DEGs.pickle
now with feature selection setup:  top50markers
Get mean and std exp!
Get common genes!  467
cell types:  10
bulk samples:  1
start BLADE!
all of 467 genes are used for optimization.
All samples are used during the optimization.
Initialization with Support vector regression
No

In [ ]:
# python "/home/cke/runscripts/Pipeline_Deconv_Run.py" "/home/cke/Puram/Puram_scanpy.h5ad" "Real" "/home/cke/Real/InputToWrapper/" "/home/cke/Real/TCGAResults/" "/home/cke/Puram/scRNAlabels/"  --name "fullpipeline_Aug1_noFS"